In [9]:
import pygame
import random

pygame.init()

screen = pygame.display.set_mode((600, 400))
pygame.display.set_caption("поймай шарик")
screen_width = 600
screen_height = 400

# Загружаем картинки
background_day = pygame.image.load("photo_5445121639533640167_y.jpg")
background_day = pygame.transform.scale(background_day, (screen_width, screen_height))
background = background_day

#звук для пропущенных шаров
miss_sound = pygame.mixer.Sound("light-dull-crunch-of-a-branch.mp3") 

running = True
clock = pygame.time.Clock()

balls = []  # список активных шаров (координаты, направление)
spawn_interval = 1000  # интервал появления шаров (миллисекунды)
last_spawn_time = 0

#корзина
player_radius = 20
player_x = screen_width // 2
player_y = screen_height - player_radius - 10  # фиксированная позиция внизу

# rjhpbyrf
def draw_basket(surface, x, y, width=60, height=30):
     # Основная часть
    basket_rect = pygame.Rect(x - width//2, y - height//2, width, height)
    pygame.draw.ellipse(surface, (139, 69, 19), basket_rect)  
    # Ручка 
    handle_points = [
        (x - width//3, y - height//2 - 5),
        (x - width//6, y - height//2 - 15),
        (x + width//6, y - height//2 - 15),
        (x + width//3, y - height//2 - 5)
    ]
    pygame.draw.lines(surface, (101, 67, 33), False, handle_points, 3)

# Счётчики
score = 0
missed_count = 0  #счётчик пропущенных шаров
max_misses = 3    #макс кол-во пропусков

#ihban
font = pygame.font.SysFont(None, 36)

while running:
    current_time = pygame.time.get_ticks()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Управление игроком мышью (только по горизонтали)
    mouse_x, mouse_y = pygame.mouse.get_pos()
    player_x = mouse_x
    # Ограничиваем движение игрока в пределах экрана
    player_x = max(player_radius, min(player_x, screen_width - player_radius))

    # Создаем новые шары с интервалом
    if current_time - last_spawn_time > spawn_interval:
        # Случайно выбираем сторону
        side = random.choice(["left", "right"])
        
        if side == "left":
            # Шары с левой стороны движутся вправо-вниз
            x = 0
            y = random.randint(0, 200)  # случайная начальная высота
            direction = "right"
        else:
            # Шары с правой стороны движутся влево-вниз
            x = 600
            y = random.randint(0, 200)  # случайная начальная высота
            direction = "left"
            
        balls.append([x, y, direction])
        last_spawn_time = current_time

    # Обновляем позиции шаров и проверяем столкновения
    balls_to_remove = []
    
    for ball in balls:
        x, y, direction = ball
        y += 2  #скорость падения
        
        if direction == "right":
            x += 1.5  # скорость движения вправо
        else:
            x -= 1.5  # скорость движения влево
            
        ball[0] = x
        ball[1] = y

        # Проверяем столкновение с игроком
        dx = player_x - x
        dy = player_y - y
        distance = (dx**2 + dy**2)**0.5
        
        if distance < player_radius + 15:  #радиус падающих шаров
            score += 1
            balls_to_remove.append(ball)

    # Удаляем пойманные шары
    for ball in balls_to_remove:
        if ball in balls:
            balls.remove(ball)

    # Убираем шары, которые вышли за пределы экрана и считаем пропуски
    new_balls = []
    for ball in balls:
        if ball[1] > screen_height:
            # Шар вышел за нижнюю границу - считаем пропуск
            missed_count += 1
            print(f"missed: {missed_count}/{max_misses}")
            # Воспроизводим звук пропуска
            miss_sound.play()
        elif -50 <= ball[0] <= 650 and ball[1] <= 450:
            # Шар еще в пределах экрана - оставляем его
            new_balls.append(ball)
    
    balls = new_balls

    # Проверяем условие проигрыша
    if missed_count >= max_misses:
        running = False

    # Отрисовка
    screen.blit(background, (0, 0))  # рисуем фон-картинку 
    
    # Рисуем падающие шары
    for x, y, direction in balls:
        if direction == "right":
            color = (255, 208, 160)  
        else:
            color = (255, 208, 160)  
        pygame.draw.circle(screen, color, (int(x), int(y)), 15)
    
    # Рисуем игрока (корзинку)
    draw_basket(screen, player_x, player_y)
    
    # Рисуем счётчик очков и пропусков
    score_text = font.render(f"score: {score}", True, (255, 255, 255))
    misses_text = font.render(f"missed: {missed_count}/{max_misses}", True, (255, 255, 255))
    
    screen.blit(score_text, (10, 10))
    screen.blit(misses_text, (10, 50))

    pygame.display.flip()
    clock.tick(60)

pygame.quit()

missed: 1/3
missed: 2/3
missed: 3/3
